In [ ]:
import copy
# Piece values for evaluation
piece_values = {'P': 1, 'N': 3, 'B': 3, 'R': 5, 'Q': 9, 'K': 1000}

# Initial chess board setup
initial_board = [
    ["r", "n", "b", "q", "k", "b", "n", "r"],
    ["p", "p", "p", "p", "p", "p", "p", "p"],
    [".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", ".", ".", "."],
    [".", ".", ".", ".", ".", ".", ".", "."],
    ["P", "P", "P", "P", "P", "P", "P", "P"],
    ["R", "N", "B", "Q", "K", "B", "N", "R"]
]


def evaluate_board(board):
    score = 0
    for row in board:
        for piece in row:
            if piece in piece_values:
                score += piece_values[piece]
            elif piece.lower() in piece_values:
                score -= piece_values[piece.lower()]
    return score

def generate_legal_moves(board, is_white):
    moves = []


    for row in range(8):
        for col in range(8):
            piece = board[row][col]

            if is_white and piece == "P":
                if row > 0 and board[row-1][col] == ".":
                    moves.append(((row, col), (row-1, col)))

            if not is_white and piece == "p":
                if row < 7 and board[row+1][col] == ".":  # Move forward
                    moves.append(((row, col), (row+1, col)))

            if (is_white and piece == "N") or (not is_white and piece == "n"):
                knight_moves = [(2, 1), (2, -1), (-2, 1), (-2, -1), (1, 2), (1, -2), (-1, 2), (-1, -2)]
                for dr, dc in knight_moves:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < 8 and 0 <= nc < 8 and (board[nr][nc] == "." or board[nr][nc].islower() != is_white):
                        moves.append(((row, col), (nr, nc)))

    return moves

def apply_move(board, move):
    (r1, c1), (r2, c2) = move
    new_board = copy.deepcopy(board)
    new_board[r2][c2] = new_board[r1][c1]
    new_board[r1][c1] = "."
    return new_board

# Beam search implementation
def beam_search(board, beam_width, depth_limit, is_white=True):
    candidates = [(board, [], evaluate_board(board))]

    for _ in range(depth_limit):
        next_candidates = []

        for state, moves, score in candidates:
            legal_moves = generate_legal_moves(state, is_white)
            move_scores = []

            for move in legal_moves:
                new_state = apply_move(state, move)
                new_score = evaluate_board(new_state)
                move_scores.append((new_score, move, new_state))


            move_scores.sort(reverse=True, key=lambda x: x[0])
            next_candidates.extend([(m[2], moves + [m[1]], m[0]) for m in move_scores[:beam_width]])

        candidates = next_candidates[:beam_width]

    best_sequence = max(candidates, key=lambda x: x[2])
    return best_sequence[1], best_sequence[2]
beam_width = 3
depth_limit = 2
best_moves, evaluation = beam_search(initial_board, beam_width, depth_limit, is_white=True)

print("Best Move Sequence:", best_moves)
print("Evaluation Score:", evaluation)

Best Move Sequence: [((6, 0), (5, 0)), ((5, 0), (4, 0))]
Evaluation Score: 1039


In [ ]:
import random
import math

def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def total_distance(route):
    return sum(distance(route[i], route[i + 1]) for i in range(len(route) - 1)) + distance(route[-1], route[0])

def hill_climb(locations, max_iterations=1000):
    current_route = locations[:]
    random.shuffle(current_route)
    current_distance = total_distance(current_route)

    for _ in range(max_iterations):
        i, j = sorted(random.sample(range(len(locations)), 2))
        new_route = current_route[:]
        new_route[i:j + 1] = reversed(new_route[i:j + 1])
        new_distance = total_distance(new_route)

        if new_distance < current_distance:
            current_route, current_distance = new_route, new_distance

    return current_route, current_distance

locations = [(0, 0), (2, 3), (5, 4), (7, 8), (6, 1), (3, 6)]
optimized_route, total_dist = hill_climb(locations)

print("Optimized Route:", optimized_route)
print("Total Distance:", total_dist)


Optimized Route: [(6, 1), (5, 4), (7, 8), (3, 6), (2, 3), (0, 0)]
Total Distance: 24.957141036098125


In [ ]:
import random
import math

def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def total_distance(route):
    return sum(distance(route[i], route[i + 1]) for i in range(len(route) - 1)) + distance(route[-1], route[0])

def create_population(cities, size):
    return [random.sample(cities, len(cities)) for _ in range(size)]

def select_parents(population):
    return sorted(population, key=total_distance)[:2]

def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(parent1)), 2))
    child = parent1[start:end + 1] + [city for city in parent2 if city not in parent1[start:end + 1]]
    return child

def mutate(route, mutation_rate=0.1):
    if random.random() < mutation_rate:
        i, j = random.sample(range(len(route)), 2)
        route[i], route[j] = route[j], route[i]
    return route

def genetic_algorithm(cities, population_size=100, generations=500):
    population = create_population(cities, population_size)

    for _ in range(generations):
        parents = select_parents(population)
        offspring = [mutate(crossover(parents[0], parents[1])) for _ in range(population_size)]
        population = parents + offspring

    best_route = min(population, key=total_distance)
    return best_route, total_distance(best_route)

cities = [(random.randint(0, 100), random.randint(0, 100)) for _ in range(10)]
best_route, best_distance = genetic_algorithm(cities)

print("Best Route:", best_route)
print("Best Distance:", best_distance)


Best Route: [(23, 62), (7, 39), (12, 14), (70, 13), (72, 37), (73, 49), (86, 80), (73, 91), (56, 64), (42, 70)]
Best Distance: 266.0443984064507
